# Create a 10 km Square GeoJSON File Centered on BONA:

First, generate a GeoJSON file representing a 10 km square centered at the BONA coordinates. This file will define your area of interest for the data retrieval.

# Import Libraries:

NEON sites:
TEAK: 37.00583, -119.00602
SOAP:37.03337, -119.26219
HARV:42.53691, -72.17265
WREF:45.82049, -121.95191
BONA: Lat: 65.1540 Lon: -147.5026


In [6]:
%pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [7]:
%%time

import os
import geopandas as gpd
from shapely.geometry import Polygon
from geopy.distance import geodesic

CPU times: user 34 µs, sys: 0 ns, total: 34 µs
Wall time: 37.4 µs


In [8]:
%%time
# Dictionary of sites with coordinates (latitude, longitude)
camera_sites = {
    "BONA": [65.1540, -147.5026],
    "WREF": [45.8205, -121.9519],
    "DEJU": [63.8811, -145.7514],
    "RMNP": [40.2759, -105.5460],
    "TEAK": [37.0058, -119.0060]
}

buffer_size_km = 50  # Length and width of square AOI in kilometers


CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 7.63 µs


In [9]:
%%time
def generate_geojson_files(sites, buffer_size_km, output_folder="geojson_files"):
    """
    Generate GeoJSON files with bounding box polygons for specified camera sites.

    Parameters:
        sites (dict): Dictionary of site names and coordinates (latitude, longitude).
        buffer_size_km (float): Buffer size in kilometers for the bounding box.
        output_folder (str): Folder to save the GeoJSON files.
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    for site_name, coords in sites.items():
        # Calculate half the buffer distance (buffer size in each direction)
        half_buffer_km = buffer_size_km / 2

        # Calculate the corner points for the bounding box
        north = geodesic(kilometers=half_buffer_km).destination((coords[0], coords[1]), 0)
        south = geodesic(kilometers=half_buffer_km).destination((coords[0], coords[1]), 180)
        east = geodesic(kilometers=half_buffer_km).destination((coords[0], coords[1]), 90)
        west = geodesic(kilometers=half_buffer_km).destination((coords[0], coords[1]), 270)

        # Define the coordinates for the square polygon
        polygon_coords = [
            (west.longitude, south.latitude),  # Bottom-left
            (west.longitude, north.latitude),  # Top-left
            (east.longitude, north.latitude),  # Top-right
            (east.longitude, south.latitude),  # Bottom-right
            (west.longitude, south.latitude)   # Close the loop
        ]

        # Create the polygon and GeoDataFrame
        polygon = Polygon(polygon_coords)
        boundary_gdf = gpd.GeoDataFrame(index=[0], crs="EPSG:4326", geometry=[polygon])

        # Save the GeoDataFrame as a GeoJSON file
        geojson_file = os.path.join(output_folder, f"{site_name}_buffer_{buffer_size_km}km.geojson")
        boundary_gdf.to_file(geojson_file, driver="GeoJSON")

        print(f"GeoJSON file saved as '{geojson_file}' with a {buffer_size_km} km square centered on {site_name}.")

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 8.58 µs


In [10]:
%%time
generate_geojson_files(camera_sites, buffer_size_km=buffer_size_km)

GeoJSON file saved as 'geojson_files/BONA_buffer_50km.geojson' with a 50 km square centered on BONA.
GeoJSON file saved as 'geojson_files/WREF_buffer_50km.geojson' with a 50 km square centered on WREF.
GeoJSON file saved as 'geojson_files/DEJU_buffer_50km.geojson' with a 50 km square centered on DEJU.
GeoJSON file saved as 'geojson_files/RMNP_buffer_50km.geojson' with a 50 km square centered on RMNP.
GeoJSON file saved as 'geojson_files/TEAK_buffer_50km.geojson' with a 50 km square centered on TEAK.
CPU times: user 29.3 ms, sys: 3.12 ms, total: 32.5 ms
Wall time: 88.9 ms
